In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eye-images-and-hb-level/Eye_Features.xlsx
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709720371526.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709800783595.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709980654965.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709709970921.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709970267160.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709985657508.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1710049249967.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709880362990.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709789649074.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1710050083980.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1710067886551.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709880494391.jpg
/kaggle/

In [14]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import KFold
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from math import ceil

In [4]:
image_folder = '/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/'
excel_path = '/kaggle/input/eye-images-and-hb-level/Eye_Features.xlsx'

In [5]:
image_size = (512, 512)

In [6]:
df = pd.read_excel(excel_path)
print(f"Excel entries: {len(df)}")

Excel entries: 404


In [7]:
# Verify image names exist in the folder and filter
available_images = set(os.listdir(image_folder))  # List of all 412 image filenames
df = df[df['Image_Name'].isin(available_images)]  # Keep only matching entries
print(f"Filtered Excel entries (matching images): {len(df)}")

Filtered Excel entries (matching images): 404


In [8]:
def load_and_preprocess_image(image_name):
    img_path = os.path.join(image_folder, image_name)
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image {image_name} not found!")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, image_size)
    # Normalize color
    img = (img - img.min()) / (img.max() - img.min())
    return img

In [9]:
# Load only the 404 images from Excel
images = []
hemoglobin_levels = []
for index, row in df.iterrows():
    img = load_and_preprocess_image(row['Image_Name'])
    images.append(img)
    hemoglobin_levels.append(row['Hb Value'])

In [10]:
X = np.array(images)
y = np.array(hemoglobin_levels)
print(f"Loaded {X.shape[0]} images with shape {X.shape}")

Loaded 404 images with shape (404, 512, 512, 3)


In [11]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

In [12]:
def build_model():
    base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(512, 512, 3))
    base_model.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(1)
    ])
    
    # Explicitly compile with 'mae' (Keras will log 'val_mae' for validation)
    model.compile(optimizer=Adam(learning_rate=0.0001), 
                  loss='mse', 
                  metrics=['mae'])  # Ensure 'mae' is here
    return model

# EarlyStopping monitors 'val_mae'
early_stopping = EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True, mode='min')

In [15]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
fold_mae = []
fold_models = []
batch_size = 4

for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
    print(f"\nTraining Fold {fold + 1}/5")
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model = build_model()
    
    steps_per_epoch = ceil(len(X_train) / batch_size)
    
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=batch_size),
        validation_data=(X_val, y_val),  # Explicit validation data
        epochs=30,
        steps_per_epoch=steps_per_epoch,
        callbacks=[early_stopping],
        verbose=1
    )
    
    val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold + 1} MAE: {val_mae:.4f}")
    fold_mae.append(val_mae)
    fold_models.append(model)

mean_mae = np.mean(fold_mae)
print(f"\nMean MAE across 5 folds: {mean_mae:.4f}")


Training Fold 1/5
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


81/81 ━━━━━━━━━━━━━━━━━━━━ 84s 647ms/step - loss: 72.8899 - mae: 7.6141 - val_loss: 3.4368 - val_mae: 1.5355
Epoch 2/30


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 3.4368 - val_mae: 1.5355
Epoch 3/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 19s 214ms/step - loss: 5.5192 - mae: 1.8813 - val_loss: 3.6190 - val_mae: 1.6114
Epoch 4/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 3.6190 - val_mae: 1.6114
Epoch 5/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 19s 212ms/step - loss: 6.0470 - mae: 1.9607 - val_loss: 4.0388 - val_mae: 1.6911
Epoch 6/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 4.0388 - val_mae: 1.6911
Fold 1 MAE: 1.5355

Training Fold 2/5
Epoch 1/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 67s 490ms/step - loss: 66.7156 - mae: 7.3432 - val_loss: 3.0723 - val_mae: 1.4361
Epoch 2/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 3.0723 - val_mae: 1.4361
Epoch 3/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 19s 211ms/step - loss: 6.3288 - mae: 2.0581 - val_loss: 3.2027 - val_mae: 1.4662


In [17]:
best_fold = np.argmin(fold_mae)
best_model = fold_models[best_fold]
best_model.summary()

# Unfreeze layers
base_model = best_model.layers[0]
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

best_model.compile(optimizer=Adam(learning_rate=1e-5), 
                   loss='mse', 
                   metrics=['mae'])  # Recompile with 'mae'

steps_per_epoch_fine = ceil(len(X) / batch_size)

history_fine = best_model.fit(
    datagen.flow(X, y, batch_size=batch_size),
    epochs=100,
    steps_per_epoch=steps_per_epoch_fine,
    callbacks=[early_stopping],
    verbose=1
)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)          │ (None, 16, 16, 1536)        │      10,783,535 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 1536)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 512)                 │         786,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,285,428 (77.38 MB)

 Trainable params: 4,291,681 (16.37 MB)

 Non-trainable params: 7,410,383 (28.27 MB)

 Optimizer params: 8,583,364 (32.74 MB)

Epoch 1/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 54s 208ms/step - loss: 6.2515 - mae: 2.0473
Epoch 2/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 302us/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 3/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 22s 207ms/step - loss: 5.5940 - mae: 1.9218
Epoch 4/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 101us/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 5/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 23s 210ms/step - loss: 5.8315 - mae: 1.9395
Epoch 6/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 92us/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 7/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 23s 208ms/step - loss: 5.1262 - mae: 1.7728
Epoch 8/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 99us/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 9/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 23s 208ms/step - loss: 5.8137 - mae: 1.9297
Epoch 10/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 97us/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 11/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 22s 207ms/step - loss: 5.3294 - mae: 1.8903
Epoch 12/100
101/101 ━━━━━━━━━━

In [18]:
y_pred = best_model.predict(X)
mae_full = np.mean(np.abs(y_pred.flatten() - y))
print(f"MAE on full dataset: {mae_full:.4f}")

for i in range(5):
    print(f"Predicted: {y_pred[i][0]:.2f}, Actual: {y[i]:.2f}")

best_model.save("/kaggle/working/hemoglobin_model.h5")
print("Model saved as hemoglobin_model.h5")

13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step 
MAE on full dataset: 1.5506
Predicted: 11.80, Actual: 11.50
Predicted: 11.80, Actual: 11.60
Predicted: 11.80, Actual: 11.10
Predicted: 11.80, Actual: 12.10
Predicted: 11.80, Actual: 12.20
Model saved as hemoglobin_model.h5
